In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000537388s, CPU 0.000519s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00733567s, CPU 0.007334s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneStress)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)


LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [4]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [5]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f2D)

In [6]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [7]:
q1 = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 3.6823357906910687e-6; -5.003913738015893e-7;;], [0.0], Int64[], 1, :u2D)

In [8]:
FEM.showDoFResults(problem, q1, :uvec, visible=true)

1

In [9]:
q2 = FEM.solveDisplacement(problem, [load], [supp])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 3.6823357906910687e-6; -5.003913738015893e-7;;], [0.0], Int64[], 1, :u2D)

In [10]:
FEM.showDoFResults(problem, q2, :uvec)

2

In [11]:
elsupp = FEM.elasticSupport("left", kx=1e6, ky=1e6)

("left", 1.0e6, 1.0e6, 0)

In [12]:
q4 = FEM.solveDisplacement(problem, [load], [], [elsupp])

LowLevelFEM.VectorField(Matrix{Float64}[], [1.138967824112648e-6; 2.281425095890338e-7; … ; 4.706195058928785e-6; -5.01577382158046e-7;;], [0.0], Int64[], 1, :u2D)

In [13]:
FEM.showDoFResults(problem, q4, :uvec)

3

In [14]:
q3 = copy(q2)
q3.a[1] = 5
q2.a[1] = 3
q3.a[1]

5.0

In [15]:
q2 + q3
q2 - q3
q2 * 2
2 * q2
q2 / 2

LowLevelFEM.VectorField(Matrix{Float64}[], [1.5; 0.0; … ; 1.8411678953455343e-6; -2.5019568690079463e-7;;], [0.0], Int64[], 1, :u2D)

In [16]:
A1 = FEM.solveStrain(problem, q1)

LowLevelFEM.TensorField([[5.012303383292967e-6; -8.973864043055219e-8; … ; 0.0; -1.4883415359807896e-6;;], [5.018225207404288e-6; -1.0057662027384346e-7; … ; 0.0; -1.4920664199839504e-6;;], [5.039404666537472e-6; 5.731650329945527e-8; … ; 0.0; -1.4910217504842391e-6;;], [4.759118649841318e-6; -4.722950788291871e-7; … ; 0.0; -1.7209469432103706e-6;;], [5.026446288166567e-6; 4.424293245154461e-8; … ; 0.0; -1.4916657475601418e-6;;], [4.563490608043547e-6; -2.0587704497835933e-7; … ; 0.0; -1.7269046851085198e-6;;], [5.041076337414415e-6; -1.7652299919550645e-8; … ; 0.0; -1.4733912984116656e-6;;], [4.892474677915217e-6; 9.457634380193556e-7; … ; 0.0; -1.7427038191585938e-6;;], [4.999711244861671e-6; 1.9281720303763613e-7; … ; 0.0; -1.5280886003357257e-6;;], [4.597600845365214e-6; 6.856141676073498e-7; … ; 0.0; -1.7210692196345666e-6;;]  …  [5.009436437674425e-6; -5.227709925497364e-7; … ; 0.0; -1.6191852905909837e-6;;], [4.983109695225119e-6; -8.138365365930829e-8; … ; 0.0; -1.4844984328389

In [17]:
A2 = FEM.solveStrain(problem, q1, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [5.309073597201354e-6; 2.199325672808693e-6; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :e)

In [18]:
FEM.showStrainResults(problem, A1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


4

In [19]:
FEM.showDoFResults(problem, A2, :e)

5

In [20]:
A3 = FEM.elementsToNodes(problem, A1)

LowLevelFEM.TensorField(Matrix{Float64}[], [5.309073597201354e-6; 2.199325672808693e-6; … ; 0.0; -1.4910719807924995e-6;;], [0.0], Int64[], 1, :e)

In [21]:
FEM.showDoFResults(problem, A3, :e)

6

In [22]:
S1 = FEM.solveStress(problem, q1)

LowLevelFEM.TensorField([[1.0000992183498087; -0.006902972340811707; … ; 0.0; 0.0;;], [1.0015833272904981; -0.007736663097987958; … ; 0.0; 0.0;;], [1.0046809872341094; 0.00440896179226579; … ; 0.0; 0.0;;], [0.9969331681617983; -0.03633039067916823; … ; 0.0; 0.0;;], [1.0027864670348783; 0.0034033024962726623; … ; 0.0; 0.0;;], [0.9677531220233949; -0.015836695767566102; … ; 0.0; 0.0;;], [1.0022257901270892; -0.0013578692245808188; … ; 0.0; 0.0;;], [1.0207966918575093; 0.07275103369379658; … ; 0.0; 0.0;;], [1.004276899261138; 0.014832092541356624; … ; 0.0; 0.0;;], [0.972103086923043; 0.052739551354411524; … ; 0.0; 0.0;;]  …  [1.0197879581946783; -0.04021315327305664; … ; 0.0; 0.0;;], [0.9950166350867765; -0.006260281050716022; … ; 0.0; 0.0;;], [0.9956815962680899; 0.006021435501200205; … ; 0.0; 0.0;;], [1.054251951123328; -0.11418056319874804; … ; 0.0; 0.0;;], [0.9972901296216864; 0.010334249008204118; … ; 0.0; 0.0;;], [0.9970074334220527; 0.010119399283299213; … ; 0.0; 0.0;;], [1.0570912

In [23]:
S2 = FEM.solveStress(problem, q1, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [1.1303639290764707; 0.16917889790836102; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :s)

In [24]:
FEM.showStressResults(problem, S1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


7

In [25]:
FEM.showDoFResults(problem, S2, :s)

8

In [26]:
S3 = FEM.elementsToNodes(problem, S1)

LowLevelFEM.TensorField(Matrix{Float64}[], [1.1303639290764707; 0.16917889790836102; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :s)

In [27]:
FEM.showDoFResults(problem, S3, :s)

9

In [28]:
thermo = FEM.Problem([mat], type=:PlaneHeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)
Info    : Renumbering nodal model data (9 steps)


LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [29]:
temp = FEM.temperatureConstraint("left", T=0)
flux = FEM.heatFlux("right", qn=1)

flx = FEM.heatFluxVector(thermo, [flux])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; -0.062499999999999924; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn)

In [30]:
KT = FEM.heatConductionMatrix(thermo)

FEM.applyBoundaryConditions!(thermo, KT, flx, [temp])

T1 = FEM.solveTemperature(KT, flx)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; -0.02222222222222229; … ; -0.005781351831909781; -0.016643653924583945;;], [0.0], Int64[], 1, :T)

In [31]:
FEM.showDoFResults(thermo, T1, :T)

10

In [32]:
T2 = FEM.solveTemperature(thermo, [flux], [temp])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; -0.02222222222222229; … ; -0.005781351831909781; -0.016643653924583945;;], [0.0], Int64[], 1, :T)

In [33]:
FEM.showDoFResults(thermo, T2, :T)

11

In [34]:
qT1 = FEM.solveHeatFlux(thermo, T2)

LowLevelFEM.VectorField([[0.9999999999999989; -1.2490009027033011e-15; … ; -1.2490009027033011e-15; 0.0;;], [0.9999999999999973; 1.2490009027033011e-15; … ; 1.2490009027033011e-15; 0.0;;], [1.0000000000000002; -8.118505867571457e-15; … ; -8.118505867571457e-15; 0.0;;], [1.0000000000000024; -4.683753385137379e-16; … ; -4.683753385137379e-16; 0.0;;], [1.000000000000004; -2.4980018054066022e-15; … ; -2.4980018054066022e-15; 0.0;;], [1.000000000000003; 7.806255641895632e-17; … ; 7.806255641895632e-17; 0.0;;], [0.9999999999999989; 7.494005416219807e-15; … ; 7.494005416219807e-15; 0.0;;], [1.000000000000004; -3.122502256758253e-16; … ; -3.122502256758253e-16; 0.0;;], [1.0000000000000042; -7.806255641895632e-16; … ; -7.806255641895632e-16; 0.0;;], [1.0000000000000044; -3.122502256758253e-16; … ; -3.122502256758253e-16; 0.0;;]  …  [1.0000000000000024; -3.1225022567582528e-15; … ; -3.1225022567582528e-15; 0.0;;], [1.000000000000003; 0.0; … ; 0.0; 0.0;;], [0.9999999999999977; 1.5612511283791264e

In [35]:
FEM.showElementResults(thermo, qT1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


12

In [36]:
qT2 = FEM.solveHeatFlux(thermo, T2, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [1.000000000000004; 7.806255641895632e-17; … ; 1.0000000000000027; 1.0408340855860843e-15;;], [0.0], Int64[], 1, :q2D)

In [37]:
FEM.showDoFResults(thermo, qT2, :qvec)

13

In [38]:
qT3 = FEM.elementsToNodes(thermo, qT1)

LowLevelFEM.VectorField(Matrix{Float64}[], [1.000000000000004; 7.806255641895632e-17; … ; 1.0408340855860843e-15; 0.0;;], [0.0], Int64[], 1, :q2D)

In [39]:
FEM.showDoFResults(thermo, qT3, :qvec)

14

In [40]:
heatconvB = FEM.heatConvection("bottom", h=1, Tₐ=0)
heatconvT = FEM.heatConvection("top", h=100, Tₐ=20)

("top", 100, 20, 0 + 2im)

In [41]:
T3 = FEM.solveTemperature(thermo, [], [temp], [heatconvB, heatconvT])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 6.028905064944964; … ; 6.149721965107743; 10.661603694875508;;], [0.0], Int64[], 1, :T)

In [42]:
FEM.showDoFResults(thermo, T3, :T)

15

In [43]:
fT = FEM.thermalLoadVector(problem, T1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.00029049635220987537; 0.00019841269841269923; … ; 0.0007247225068676066; 4.336808689942018e-19;;], [0.0], Int64[], 1, :f2D)

In [44]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])
q5 = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 3.6823357906910687e-6; -5.003913738015893e-7;;], [0.0], Int64[], 1, :u2D)

In [45]:
ST1 = FEM.solveStress(problem, q5, T=T1)

LowLevelFEM.TensorField([[1.058359580269888; -0.006902972340811707; … ; 0.0; 0.0;;], [1.0695259424925514; -0.007736663097987958; … ; 0.0; 0.0;;], [1.071632671732965; 0.00440896179226579; … ; 0.0; 0.0;;], [1.0059436584552681; -0.03633039067916823; … ; 0.0; 0.0;;], [1.0603722838087484; 0.0034033024962726623; … ; 0.0; 0.0;;], [0.9767636123168648; -0.015836695767566102; … ; 0.0; 0.0;;], [1.0784162663175658; -0.0013578692245808188; … ; 0.0; 0.0;;], [1.0291384207861727; 0.07275103369379658; … ; 0.0; 0.0;;], [1.0278877071517016; 0.014832092541356624; … ; 0.0; 0.0;;], [0.985655332208538; 0.052739551354411524; … ; 0.0; 0.0;;]  …  [1.0340736724803927; -0.04021315327305664; … ; 0.0; 0.0;;], [1.0568810410936256; -0.006260281050716022; … ; 0.0; 0.0;;], [1.0626705839432378; 0.006021435501200205; … ; 0.0; 0.0;;], [1.0685376654090424; -0.11418056319874804; … ; 0.0; 0.0;;], [1.0495477570810272; 0.010334249008204118; … ; 0.0; 0.0;;], [1.0446264810411006; 0.010119399283299213; … ; 0.0; 0.0;;], [1.0654329

In [46]:
ST2 = FEM.solveStress(problem, q5, T=T1, T₀=T1)

LowLevelFEM.TensorField([[1.0000992183498087; -0.006902972340811707; … ; 0.0; 0.0;;], [1.0015833272904981; -0.007736663097987958; … ; 0.0; 0.0;;], [1.0046809872341094; 0.00440896179226579; … ; 0.0; 0.0;;], [0.9969331681617983; -0.03633039067916823; … ; 0.0; 0.0;;], [1.0027864670348783; 0.0034033024962726623; … ; 0.0; 0.0;;], [0.9677531220233949; -0.015836695767566102; … ; 0.0; 0.0;;], [1.0022257901270892; -0.0013578692245808188; … ; 0.0; 0.0;;], [1.0207966918575093; 0.07275103369379658; … ; 0.0; 0.0;;], [1.004276899261138; 0.014832092541356624; … ; 0.0; 0.0;;], [0.972103086923043; 0.052739551354411524; … ; 0.0; 0.0;;]  …  [1.0197879581946783; -0.04021315327305664; … ; 0.0; 0.0;;], [0.9950166350867765; -0.006260281050716022; … ; 0.0; 0.0;;], [0.9956815962680899; 0.006021435501200205; … ; 0.0; 0.0;;], [1.054251951123328; -0.11418056319874804; … ; 0.0; 0.0;;], [0.9972901296216864; 0.010334249008204118; … ; 0.0; 0.0;;], [0.9970074334220527; 0.010119399283299213; … ; 0.0; 0.0;;], [1.0570912

In [47]:
FEM.showElementResults(problem, ST1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


16

In [48]:
FEM.showElementResults(problem, ST2, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


17

In [49]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [50]:
gmsh.finalize()